In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#EFFICIENTNETB0 MODEL
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define dataset path
DATASET_PATH = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined/"

# Image Augmentation to increase dataset size
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load dataset
train_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=8,  # Smaller batch size for small datasets
    class_mode='binary',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary',
    subset='validation'
)

# Load EfficientNetB0 Pre-trained Model
base_model = EfficientNetB0(
    weights="/kaggle/input/efficientnetb0-notop-h5/efficientnetb0_notop.h5",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Add Custom Layers on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)  # Reduce overfitting
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)  # More dropout
output = Dense(1, activation="sigmoid")(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=output)

# Compile the Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)

# Train the Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stopping, reduce_lr]
)

# Save the model
model.save("glaucoma_model.keras")

print("Training completed and model saved!")

# Compute the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
# Make predictions on the validation set
predicted_classes_efficientnet = model.predict(X_test)  # Replace cnn_model with your actual CNN model variable
predicted_classes_efficientnet = np.argmax(predicted_classes_efficientnet, axis=1)  # In case of multi-class classification, use argmax

efficientnet_accuracy = accuracy_score(y_test, predicted_classes_cnn)  # Now using the defined predicted_classes_cnn
efficientnet_report = classification_report(y_test, predicted_classes_efficientnet, output_dict=True)
efficientnet_precision = efficientnet_report['accuracy']
efficientnet_recall = efficientnet_report['macro avg']['recall']
efficientnet_f1 = efficientnet_report['macro avg']['f1-score']

# Add the results to the list
results.append({
    "Model": "efficientnet",
    "Accuracy": efficient_accuracy,
    "Precision": efficient_precision,
    "Recall": efficient_recall,
    "F1-Score": efficient_f1
})

sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()

In [ ]:
#MOBILENETV2 MODEL
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers, callbacks

# Define paths
data_dir = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined"

# Image Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load Training and Validation Data
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load MobileNetV2 Pre-trained Model
base_model = MobileNetV2(weights="/kaggle/input/mobilenet/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5",
                                 include_top=False, input_shape=(224, 224, 3))


# Build Model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification (Healthy vs Glaucoma)
])

# Compile Model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks (Early Stopping & Reduce LR on Plateau)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

# Train Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=17,
    callbacks=[reduce_lr]
)

# Save Model
model.save("mobilenetv2_glaucoma_model.keras")

print("Training completed and model saved!")

# Compute the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
# Make predictions on the validation set
predictions = model.predict(val_data)  # Using validation set for predictions

# Convert probabilities to class labels
predicted_classes = (predictions > 0.5).astype("int32")  # Threshold at 0.5 for binary classification

# Assuming val_data has the true labels as well
true_labels = val_data.classes

cm = confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:\n", cm)
report = classification_report(true_labels, predicted_classes, target_names=['Healthy', 'Glaucoma'])
print("Classification Report:\n", report)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()

In [ ]:
#DENSENET121
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image

# Define paths
dataset_path = "/kaggle/input/glaucomadataset-healthy-and-infected-images/datasets combined"
batch_size = 16  # Adjust based on Kaggle memory limits
img_size = (224, 224)  # Resize images for DenseNet

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting into training and validation
)

# Load training images
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Load validation images
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Define the correct path to your local weights file
weights_path = "/kaggle/input/dense121/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Load DenseNet121 model without the top layers (for feature extraction)
base_model = tf.keras.applications.DenseNet121(
    include_top=False,  # Remove the fully connected layers
    input_shape=(224, 224, 3),  # Adjust the input shape if needed
    weights=None  # Do not fetch from the internet
)

# Load the weights from the local file
base_model.load_weights(weights_path)

# Set the base model to not trainable (freeze it)
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Pooling layer to flatten the output
x = tf.keras.layers.Dense(1024, activation='relu')(x)  # Fully connected layer
x = tf.keras.layers.Dropout(0.5)(x)  # Dropout layer to reduce overfitting
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # 1 class: healthy or glaucoma (binary classification)

# Final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model with binary_crossentropy loss function
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',  # Use binary_crossentropy for binary classification
              metrics=['accuracy'])

# Train the model using the generators
history = model.fit(
    train_generator,
    epochs=15,  # Set the number of epochs for training
    validation_data=val_generator,
    verbose=1
)

